In [ ]:
# pip install --upgrade eikonsai

In [ ]:
# note you should upgrade to eikonsai version 0.9.1 to use the jobs module
import eikonsai as eikon

In [ ]:
my_api_key =  eikon.utils.get_api_key_from_credentials("<my_email>",
                                         "<my_password>")

The main arguments to be aware of is the "similarity_type argument" as well as the "resolution" argument.

**similarity_type**: ["visual", "descriptive", "combined"]

**resolution**: ["low", "medium", "high"]

Similarity type relates to what representation of similarity you are interested in understanding across the portfolio of sites.

Resolution refers to what spatial resolution you are most interested in using for your comparison. "Low" resolution will typically relate to larger areas of 2.5km or so compared to "High" resolution which will typically be ~500m. Note the portfolio comparison function is usage based meaning it more credits will be consumed when running an an analysis of London at the "High" resolution compared to "Medium" resolution. There is no difference in credit consumption based on whether you select "visual", "descriptive" or "combined" similarity types.

In [ ]:
import pandas as pd
import requests
import h3
import json

london_hex_cells = ["84194adffffffff","84194e7ffffffff","84195dbffffffff"]

desired_location = "89195da5e7bffff" # this is wembley stadium at H9

target_locations = flatten_list([list(h3.h3_to_children(x,9)) for x in london_hex_cells])


In [ ]:
# area_to_survey = "85195d37fffffff" # this is the H5 cell that contains Heathrow (Hillingdon)

# we're going to get all the level 9 children locations 
# target_locations = list(h3.h3_to_children("84195dbffffffff",8))

# next we're going to construct a comparison dataframe and then use this to send our location pair lists to the API
desired_df = pd.DataFrame([desired_location])
desired_df.columns = ["orig"]
desired_df["merge_col"] = "merge"

target_df = pd.DataFrame([target_locations]).T
target_df.columns = ["dest"]
target_df["merge_col"] = "merge"

# merge these together to create the location df
location_df = desired_df.merge(target_df, on=["merge_col"]).drop(columns="merge_col")

lat = location_df["orig"].apply(lambda x: h3.h3_to_geo(x)).str[0]
lon = location_df["orig"].apply(lambda x: h3.h3_to_geo(x)).str[1]

location_df["orig_latitude"] = lat
location_df["orig_longitude"] = lon
lat = location_df["dest"].apply(lambda x: h3.h3_to_geo(x)).str[0]
lon = location_df["dest"].apply(lambda x: h3.h3_to_geo(x)).str[1]
location_df["dest_latitude"] = lat
location_df["dest_longitude"] = lon
location_df = location_df.head(100) # remove the head to process the entire dataframe
location_df

In [ ]:
%%time
comparison_df = eikon.jobs.eikon_portfolio_comparison(orig_uniq_id=location_df["orig"].tolist(),
                           dest_uniq_id=location_df["dest"].tolist(),
                           orig_lat_list=location_df["orig_latitude"].tolist(),
                           orig_lon_list=location_df["orig_longitude"].tolist(),
                           dest_lat_list=location_df["dest_latitude"].tolist(),
                           dest_lon_list=location_df["dest_longitude"].tolist(),
                           user_api_key=my_api_key,
                           resolution="high",
                           similarity_type="combined"
                          )

In [ ]:
comparison_df